In [105]:
import openai
import time
import threading
from contextlib import contextmanager
import re
import os
import pickle

In [106]:
openai.api_key = <your openai apikey>

In [107]:
class TimeoutException(Exception):
    def __init__(self, msg=''):
        self.msg = msg

@contextmanager
def time_limit(seconds, msg=''):
    
    timer = threading.Timer(seconds, lambda: _thread.interrupt_main())
    timer.start()
    try:
        yield
    except KeyboardInterrupt:
        raise TimeoutException("Timed out for operation {}".format(msg))
    finally:
        # if the action ends in specified time, timer is canceled
        timer.cancel()

In [108]:
def query_openai(prompt,
                 output_tokens_number = 1024,
                 retry=2):
  for i in range(retry):
    try:
      response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        max_tokens=output_tokens_number, 
        prompt=prompt, 
        n=1, # num of returned sequence
      )
      return response["choices"][0]["text"]
    except:
      time.sleep(2)
  print(f'query_openai fail')
  return None

In [109]:
class story_generator:
    def __init__(self, id, genre, people) -> None:
        self.id = id
        self.genre = genre
        self.people = people
        self.chapter_id = 1
        self.full_chapter_outlines = ""
        self.stories = []
        self.role = f'You are a {self.genre} author. Your task is to write {self.genre} stories for {self.people} in a vivid and intriguing language.'
        action = f'Fill out the template below for a {self.genre} story for {self.people}.'
        template = f'''
        Title: [Insert story title here]

        Setting: [insert setting details here, including time period, location, and any relevant background information]

        Protagonist: [Insert protagonist's name, age, and occupation, as well as a brief description of their personality and motivations]

        Antagonist: [Insert antagonist's name, age, and occupation, as well as a brief description of their personality and motivations]

        Conflict: [Insert the main conflict of the story, including the problem the protagonist faces and the stakes involved]

        Dialog: [Instructions for using dialogue to advance the plot, reveal character, and provide information to the reader]

        Theme: [Insert the central theme of the story and instructions for developing it throughout the plot, character, and setting]

        Tone: [Insert the desired tone for the story and instructions for maintaining consistency and appropriateness to the setting and characters]

        Pacing: [Instructions for varying the pace of the story to build and release tension, advance the plot, and create dramatic effect]

        Optional: [insert and additional details or requirements for the story, such as specific word count or genre constraints]
        '''
        prompt = self.role + action + template
        self.setting = query_openai(prompt)
        print(f'story setting:\n{self.setting}')
        self.save_to_file()
        
    def save_to_file(self):
        with open(str(self.id), 'wb') as file:
            pickle.dump(self, file)
    
    @classmethod
    def load_from_file(cls, filename):
        with open(filename, 'rb') as file:
            return pickle.load(file)
    
    def write_story_to_file(self):
        with open(f'{str(self.id)}_story', "w") as file:
            for story in self.stories:
                file.write(story + "\n")
    
    def write_outline_to_file(self):
        with open(f'{str(self.id)}_story_outline', "w") as file:
            file.write(self.full_chapter_outlines + "\n")
    
    def generate_next_story(self):
        prompt = self.role + f'''
        Given the setting, build the {self.chapter_id} chapter outline.
        #### Setting ####
        {self.setting}'''
        prompt_variation = ""
        if self.full_chapter_outlines != "":
            prompt += f'''#### Previous Chapters ####
            {self.full_chapter_outlines}
            '''
            prompt_variation = " and Previous Chapters"
        prompt += f'''#### Instruction ####
        Given the Setting{prompt_variation}, build the {self.chapter_id} chapter outline with following format:
        #### format ####
        Chapter {self.chapter_id}
        event 1
        event 2
        event ...
        '''
        chapter_outline = query_openai(prompt)
        self.full_chapter_outlines += "\n" + chapter_outline
        
        prompt = f'''
        {self.role}
        #### setting ####
        {self.setting}
        #### chapter outlines ####
        {self.full_chapter_outlines} 
        #### Instruction ####
        Write chapter {self.chapter_id} in depth and in great detail, in an intriguing writing style:
        Chapter {self.chapter_id} - [Chapter Title]
        [Chapter Content]
        '''
        story = query_openai(prompt, output_tokens_number=1024)
        print(story)
        self.stories.append(story)
        self.chapter_id += 1
        self.save_to_file()
        return story
        

In [110]:
unique_id = 12345
magic_story_for_6yr = story_generator(id = unique_id, genre = "magic", people = '6 years old')

story setting:

Title: The Enchanted Garden

Setting: The story takes place in a magical garden, hidden away in a faraway land. It is always springtime in the garden, with colorful flowers and lush green trees. The garden is home to all sorts of magical creatures, such as talking animals and friendly fairies.

Protagonist: Lily, a curious and kind-hearted 6-year-old girl. She loves exploring and has a great love for nature. Lily's dream is to have her own magical powers to make the world a better place.

Antagonist: The evil witch, Hilda. She is jealous of the magic that exists in the garden and will stop at nothing to possess it for herself. Hilda's dark magic has caused the garden to slowly wither and fade away.

Conflict: Lily must use her bravery and wit to outsmart the wicked witch and save the enchanted garden from disappearing forever. The stakes are high as without the magic of the garden, all the magical creatures, including Lily's new friends, will have to leave and the land 

In [111]:
story = magic_story_for_6yr.generate_next_story()


Chapter 1 - A Sprinkle of Magic

In a lush and beautiful garden hidden away in a faraway land, a 6-year-old girl named Lily is exploring with wide eyes and a curious heart. She follows a trail of colorful flowers and discovers something she never thought possible - talking animals. A squirrel and a bird welcome her to their magical world.

Lily is amazed and asks the squirrel, "How do you talk?" The little animal responds, "Why, all the creatures in this garden have the gift of speech." Lily's eyes widen in wonder and she can't wait to explore more of this enchanting place.

As she walks further into the garden, Lily comes across a fairy with glistening wings named Flora. Flora introduces herself and tells Lily about the garden and its magical inhabitants. She also explains the importance of the garden's magic and how it's slowly fading away.

"Without the magic of this garden, all the creatures who call it home will have to leave and it will become a dull and lifeless place," Flora e

In [112]:
story = magic_story_for_6yr.generate_next_story()


Chapter 2 - A Magical Picnic in the Park

After saving the Enchanted Garden from the evil witch's grasp, Lily returned home with a newfound confidence and a spark of magic in her eyes. She couldn't wait to tell her parents about her adventure and the incredible powers she had gained.

The next day, Lily's parents suggested a family picnic to the park. Lily enthusiastically agreed, knowing that she could finally show off her new abilities. As they made their way to the park, Lily couldn't contain her excitement. She skipped ahead of her parents, her silver-blonde hair bouncing with each step.

As they arrived at the park, they found a perfect spot under a big oak tree. Lily laid out the checkered picnic blanket and her parents unpacked the delicious sandwiches and snacks they had brought. As they enjoyed their lunch, Lily's parents asked her about her adventure in the Enchanted Garden.

Lily eagerly told them everything, from meeting the talking animals and fairies to her epic battle a

In [113]:
magic_story_for_6yr.write_story_to_file()

In [114]:
magic_story_for_6yr.write_outline_to_file()